# Chapter 7 : Signal Processing and Time Series

In [ ]:
import pkgutil as pu
import numpy as np
import matplotlib as mpl
import scipy as sp
import pandas as pd
import pydoc
import matplotlib.pyplot as plt
import statsmodels.api as sm
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=15,6

# Introduction to Time Series

## Stationary and non Stationary Time Series

### Sunspots data

In [ ]:
print(sm.datasets.sunspots.NOTE)

In [ ]:
sunspots_df = sm.datasets.sunspots.load_pandas().data
del sunspots_df["YEAR"]
sunspots_sa = sunspots_df["SUNACTIVITY"]

In [ ]:
sunspots_df.head(5)

In [ ]:
sunspots_df.plot(figsize=(12,8))
plt.show()

### US Macroeconomic data

In [ ]:
print(sm.datasets.macrodata.NOTE)

In [ ]:
macro_df = sm.datasets.macrodata.load_pandas().data

In [ ]:
macro_df.head(5)

In [ ]:
macro_df.index = pd.Index(sm.tsa.datetools.dates_from_range('1959Q1', '2009Q3'))
macro_cpi = macro_df["cpi"]

In [ ]:
macro_cpi.plot(figsize=(12,8))
plt.show()

### Airline dataset

In [ ]:
airline = pd.Series.from_csv('international-airline-passengers-cleaned.csv', header=0)

In [ ]:
airline.head(5)

In [ ]:
airline.plot()
plt.show()

## Checking Data types

In [ ]:
print(sunspots_df.dtypes)

In [ ]:
print(macro_df.dtypes)

In [ ]:
print(airline.dtypes)

In [ ]:
print(type(sunspots_df))

In [ ]:
print(type(macro_df))

In [ ]:
print(type(macro_cpi))

In [ ]:
print(type(airline))

# Visualize Time Series

In [ ]:
sunspots_df.plot(style="b.")
plt.show()

In [ ]:
macro_cpi.plot(style="r.")
plt.show()

In [ ]:
macro_df.plot(style=["b.","r.","g."])
plt.show()

In [ ]:
airline.plot(style="g.")
plt.show()

In [ ]:
sunspots_df.plot(kind="kde")
plt.show()

In [ ]:
macro_cpi.plot(kind="kde")
plt.show()

In [ ]:
macro_df.plot(kind="kde")
plt.show()

In [ ]:
airline.plot(kind="kde")
plt.show()

In [ ]:
sunspots_df.hist()
plt.show()

In [ ]:
macro_cpi.hist()
plt.show()

In [ ]:
macro_df.hist()
plt.show()

In [ ]:
macro_df.hist(layout=(7,2),figsize=(12,20))
plt.show()

In [ ]:
airline.hist()
plt.show()

In [ ]:
pd.tools.plotting.lag_plot(sunspots_df)
plt.show()

In [ ]:
pd.tools.plotting.lag_plot(macro_cpi)
plt.show()

In [ ]:
pd.tools.plotting.lag_plot(airline)
plt.show()

# Check stationarity

## Dickey-Fuller Test
### Null Hypothesis (H0): If accepted, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.
### Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12,center=False).mean()
    rolstd = timeseries.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(sunspots_sa)

In [ ]:
test_stationarity(macro_cpi)

In [ ]:
test_stationarity(airline)

In [ ]:
airline_log = np.log(airline)

In [ ]:
test_stationarity(airline_log)

# Decomposing TimeSeries

## Multiplicative Model : y(t) = Level * Trend * Seasonality * Noise

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(airline, model='multiplicative')
result.plot()
plt.show()

## Additive Model : y(t) = Level + Trend + Seasonality + Noise

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(macro_cpi, model='additive')
result.plot()
plt.show()

# Smoothing with moving average

In [ ]:
plt.plot(sunspots_sa, label="Original")
plt.plot(sunspots_sa.rolling(window=11).mean(), label="SMA 11")
plt.plot(sunspots_sa.rolling(window=22).mean(), label="SMA 22")
plt.legend()
plt.show()

In [ ]:
plt.plot(airline, label="Original")
plt.plot(airline.rolling(window=11).mean(), label="SMA 11")
plt.plot(airline.rolling(window=22).mean(), label="SMA 22")
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame({'SUNACTIVITY':sunspots_df['SUNACTIVITY']})
ax = df.plot()

def plot_window(win_type):
    df2 = df.rolling(window=11, win_type=win_type).mean()
    df2.columns = [win_type]
    df2.plot(ax=ax)

plot_window('boxcar')
plot_window('triang')
plot_window('blackman')
plot_window('hanning')
plot_window('bartlett')
plt.show()

ax = df.plot()

def plot_window(win_type):
    df2 = df.rolling(window=22, win_type=win_type).mean()
    df2.columns = [win_type]
    df2.plot(ax=ax)

plot_window('boxcar')
plot_window('triang')
plot_window('blackman')
plot_window('hanning')
plot_window('bartlett')
plt.show()

# Cointegration

In [ ]:
import statsmodels.api as sm
from pandas.stats.moments import rolling_window
import pandas as pd
import statsmodels.tsa.stattools as ts
import numpy as np

def calc_adf(x, y):
    result = sm.OLS(x, y).fit()    
    return ts.adfuller(result.resid)

data_loader = sm.datasets.sunspots.load_pandas()
data = data_loader.data.values
N = len(data)

t = np.linspace(-2 * np.pi, 2 * np.pi, N)
sine = np.sin(np.sin(t))
print("Self ADF", calc_adf(sine, sine))

noise = np.random.normal(0, .01, N)
print("ADF sine with noise", calc_adf(sine, sine + noise))

cosine = 100 * np.cos(t) + 10
print("ADF sine vs cosine with noise", calc_adf(sine, cosine + noise))

print("Sine vs sunspots", calc_adf(sine, data))

# Autocorrelation

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot

data_loader = sm.datasets.sunspots.load_pandas()
data = data_loader.data["SUNACTIVITY"].values
y = data - np.mean(data)
norm = np.sum(y ** 2)
correlated = np.correlate(y, y, mode='full')/norm
res = correlated[int(len(correlated)/2):]

print(np.argsort(res)[-5:])
plt.plot(res)
plt.grid(True)
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.show()
autocorrelation_plot(data)
plt.show()

# Autoregressive models

In [ ]:
from scipy.optimize import leastsq
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

def model(p, x1, x10):
   p1, p10 = p
   return p1 * x1 + p10 * x10

def error(p, data, x1, x10):
   return data - model(p, x1, x10)

def fit(data):
   p0 = [.5, 0.5]
   params = leastsq(error, p0, args=(data[10:], data[9:-1], 
data[:-10]))[0]
   return params

data_loader = sm.datasets.sunspots.load_pandas()
sunspots = data_loader.data["SUNACTIVITY"].values

cutoff = int(.9 * len(sunspots))
params = fit(sunspots[:cutoff])
print("Params", params)

pred = params[0] * sunspots[cutoff-1:-1] + params[1] * sunspots[cutoff-10:-10]
actual = sunspots[cutoff:]
print("Root mean square error", np.sqrt(np.mean((actual - pred) ** 2)))
print("Mean absolute error", np.mean(np.abs(actual - pred)))
print("Mean absolute percentage error", 100 * 
np.mean(np.abs(actual - pred)/actual))
mid = (actual + pred)/2
print("Symmetric Mean absolute percentage error", 100 * 
np.mean(np.abs(actual - pred)/mid))
print("Coefficient of determination", 1 - ((actual - pred) ** 
2).sum()/ ((actual - actual.mean()) ** 2).sum())
year_range = data_loader.data["YEAR"].values[cutoff:]
plt.plot(year_range, actual, 'o', label="Sunspots")
plt.plot(year_range, pred, 'x', label="Prediction")
plt.grid(True)
plt.xlabel("YEAR")
plt.ylabel("SUNACTIVITY")
plt.legend()
plt.show()

# ARMA models

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
years = df["YEAR"].values.astype(int)
df.index = pd.Index(sm.tsa.datetools.dates_from_range(str(years[0]), 
str(years[-1])))
del df["YEAR"]

model = sm.tsa.ARMA(df, (10,1)).fit()
prediction = model.predict('1975', str(years[-1]), dynamic=True)

df['1975':].plot()
prediction.plot(style='--', label='Prediction')
plt.legend()
plt.show()

# Periodc signals

In [ ]:
from scipy.optimize import leastsq
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
def model(p, t):
   C, p1, f1, phi1 , p2, f2, phi2, p3, f3, phi3 = p
   return C + p1 * np.sin(f1 * t + phi1) + p2 * np.sin(f2 * t + 
phi2) +p3 * np.sin(f3 * t + phi3)


def error(p, y, t):
   return y - model(p, t)

def fit(y, t):
   p0 = [y.mean(), 0, 2 * np.pi/11, 0, 0, 2 * np.pi/22, 0, 0, 2 * 
np.pi/100, 0]
   params = leastsq(error, p0, args=(y, t))[0]
   return params

data_loader = sm.datasets.sunspots.load_pandas()
sunspots = data_loader.data["SUNACTIVITY"].values
years = data_loader.data["YEAR"].values

cutoff = int(.9 * len(sunspots))
params = fit(sunspots[:cutoff], years[:cutoff])
print("Params", params)

pred = model(params, years[cutoff:])
actual = sunspots[cutoff:]
print("Root mean square error", np.sqrt(np.mean((actual - pred) ** 
2)))
print("Mean absolute error", np.mean(np.abs(actual - pred)))
print("Mean absolute percentage error", 100 * 
np.mean(np.abs(actual - pred)/actual))
mid = (actual + pred)/2
print("Symmetric Mean absolute percentage error", 100 * 
np.mean(np.abs(actual - pred)/mid))
print("Coefficient of determination", 1 - ((actual - pred) ** 
2).sum()/ ((actual - actual.mean()) ** 2).sum())
year_range = data_loader.data["YEAR"].values[cutoff:]
plt.plot(year_range, actual, 'o', label="Sunspots")
plt.plot(year_range, pred, 'x', label="Prediction")
plt.grid(True)
plt.xlabel("YEAR")
plt.ylabel("SUNACTIVITY")
plt.legend()
plt.show()

# Fourier Analysis

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.fftpack import rfft
from scipy.fftpack import fftshift

data_loader = sm.datasets.sunspots.load_pandas()
sunspots = data_loader.data["SUNACTIVITY"].values

t = np.linspace(-2 * np.pi, 2 * np.pi, len(sunspots))
mid = np.ptp(sunspots)/2
sine = mid + mid * np.sin(np.sin(t))

sine_fft = np.abs(fftshift(rfft(sine)))
print("Index of max sine FFT", np.argsort(sine_fft)[-5:])

transformed = np.abs(fftshift(rfft(sunspots)))
print("Indices of max sunspots FFT", np.argsort(transformed)[-5:])

plt.subplot(311)
plt.plot(sunspots, label="Sunspots")
plt.plot(sine, lw=2, label="Sine")
plt.grid(True)
plt.legend()
plt.subplot(312)
plt.plot(transformed, label="Transformed Sunspots")
plt.grid(True)
plt.legend()
plt.subplot(313)
plt.plot(sine_fft, lw=2, label="Transformed Sine")
plt.grid(True)
plt.legend()
plt.show()

# Spectral Analysis

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.fftpack import rfft
from scipy.fftpack import fftshift

data_loader = sm.datasets.sunspots.load_pandas()
sunspots = data_loader.data["SUNACTIVITY"].values

transformed = fftshift(rfft(sunspots))

plt.subplot(311)
plt.plot(sunspots, label="Sunspots")
plt.legend()
plt.subplot(312)
plt.plot(transformed ** 2, label="Power Spectrum")
plt.legend()
plt.subplot(313)
plt.plot(np.angle(transformed), label="Phase Spectrum")
plt.grid(True)
plt.legend()
plt.show()

# Filtering

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from scipy.signal import wiener
from scipy.signal import detrend

data_loader = sm.datasets.sunspots.load_pandas()
sunspots = data_loader.data["SUNACTIVITY"].values
years = data_loader.data["YEAR"].values

plt.plot(years, sunspots, label="SUNACTIVITY")
plt.plot(years, medfilt(sunspots, 11), lw=2, label="Median")
plt.plot(years, wiener(sunspots, 11), '--', lw=2, label="Wiener")
plt.plot(years, detrend(sunspots), lw=3, label="Detrend")
plt.xlabel("YEAR")
plt.grid(True)
plt.legend()
plt.show()